In [1]:
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from pandas import DataFrame
import pandas as pd
import seaborn as sns
import numpy as np
import json
import os
import tensorflow as tf
from sklearn.preprocessing import StandardScaler,LabelEncoder,LabelEncoder

np.set_printoptions(threshold=np.inf)

fileName1 = './data/CMU_exp_shrink_output_rep50.xls'
df_ora1 = pd.read_excel(fileName1)

fileName2 = './data/CMU_oral_output.xls'
df_ora2 = pd.read_excel(fileName2)

df_ora = pd.merge(df_ora1,df_ora2,on = 'key')
#df_ora = df_ora1

#object_user = 's002'
object_user = 's032'
#object_user = 's036'
#object_user = 's047'
#object_user = 's052'

df_ora.loc[df_ora['user']!=object_user,'user'] = 'imposter'
df = df_ora
#df = df_ora.loc[(df_ora['user']==object_user) | (df_ora['session']<400)]


le = LabelEncoder()
df['target'] = le.fit_transform(df['user'])
df.drop(['user'], axis=1, inplace=True)

X = df.drop(['target','session','key','subject','rep','sessionIndex','adj29'], axis=1)
y = df['target'].values

ss = StandardScaler()
X = ss.fit_transform(X)
X = pd.DataFrame(X)

将样本分为训练集和测试集

In [2]:
'''
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=19, shuffle=True)
X_train.shape, X_test.shape, y_train.shape, y_test.shape
'''
'''
#s002
index_train1 = [i for i in range(200)]    #s002
index_train2 = [i+400 for i in range(50)] #s032
index_train3 = [i+800 for i in range(50)] #s036
index_train4 = [i+1200 for i in range(50)] #s047
index_train5 = [i+1600 for i in range(50)] #s052

index_test1 = [i+200 for i in range(200)] #s002
index_test2 = [i+450 for i in range(350)]  #s032
index_test3 = [i+850 for i in range(350)]  #s036
index_test4 = [i+1250 for i in range(350)]  #s047
index_test5 = [i+1650 for i in range(350)]  #s052
'''
#s032
index_train1 = [i+400 for i in range(200)] #s032
index_train2 = [i for i in range(50)] #s002
index_train3 = [i+800 for i in range(50)] #s036
index_train4 = [i+1200 for i in range(50)] #s047
index_train5 = [i+1600 for i in range(50)] #s052

index_test1 = [i+600 for i in range(200)] #s032
index_test2 = [i+50 for i in range(350)]  #s002
index_test3 = [i+850 for i in range(350)]  #s036
index_test4 = [i+1250 for i in range(350)]  #s047
index_test5 = [i+1650 for i in range(350)]  #s052

'''
#s036
index_train1 = [i+200 for i in range(200)] #s036
index_train2 = [i for i in range(50)] #s002
index_train3 = [i+100 for i in range(50)] #s032
index_train4 = [i+600 for i in range(50)] #s047
index_train5 = [i+700 for i in range(50)] #s052

index_test1 = [i+400 for i in range(200)] #s036
index_test2 = [i+50 for i in range(50)]  #s002
index_test3 = [i+150 for i in range(50)]  #s032
index_test4 = [i+650 for i in range(50)]  #s047
index_test5 = [i+750 for i in range(50)]  #s052
'''

'''
#s047
index_train1 = [i+300 for i in range(200)] #s047
index_train2 = [i for i in range(50)] #s002
index_train3 = [i+100 for i in range(50)] #s032
index_train4 = [i+200 for i in range(50)] #s036
index_train5 = [i+700 for i in range(50)] #s052

index_test1 = [i+500 for i in range(200)] #s047
index_test2 = [i+50 for i in range(50)]  #s002
index_test3 = [i+150 for i in range(50)]  #s032
index_test4 = [i+250 for i in range(50)]  #s036
index_test5 = [i+750 for i in range(50)]  #s052
'''
'''
#s052
index_train1 = [i+400 for i in range(200)] #s052
index_train2 = [i for i in range(50)] #s002
index_train3 = [i+100 for i in range(50)] #s032
index_train4 = [i+200 for i in range(50)] #s036
index_train5 = [i+300 for i in range(50)] #s047

index_test1 = [i+600 for i in range(200)] #s052
index_test2 = [i+50 for i in range(50)]  #s002
index_test3 = [i+150 for i in range(50)]  #s032
index_test4 = [i+250 for i in range(50)]  #s036
index_test5 = [i+350 for i in range(50)]  #s047
'''

index_train = index_train1 + index_train2 + index_train3 + index_train4 + index_train5
index_test = index_test1 + index_test2 + index_test3 + index_test4 + index_test5

X_train = X.iloc[index_train]
X_test = X.iloc[index_test]
y_train = y[index_train]
y_test = y[index_test]

度量选择（AUC）

SVM2

In [3]:
from numpy.random import seed
seed(1)
tf.random.set_seed(2)

import numpy as np
import sklearn.model_selection as ms
import sklearn.svm as svm
import sklearn.metrics as sm
import matplotlib.pyplot as plt

# 最优超参数组合列表
params = [
        {'kernel': ['linear'], 'C': [1, 10, 100, 1000]},
        {'kernel': ['poly'], 'C': [1,10,100], 'degree': [2, 3]},
        {'kernel': ['rbf'], 'C': [1, 10, 100, 100,1000], 'gamma':[1, 0.1, 0.01, 0.001]}
        ]

model = ms.GridSearchCV(svm.SVC(probability=True), 
                        params, 
                        refit=True,
                        return_train_score=True,        # 后续版本需要指定True才有score方法  
                        cv=4)
model.fit(X_train, y_train)

# GridSearchCV的属性
print('Attrabutes:vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv')
print('cv_results_:',model.cv_results_.keys())
print('Desc:',model.cv_results_['params'][2], model.cv_results_['mean_train_score'][2], 
      model.cv_results_['mean_test_score'][2],model.cv_results_['rank_test_score'][2])
print('best_estimator_:',model.best_estimator_) 
print('best_params_:',model.best_params_) 
print('best_params_:', model.cv_results_['params'][model.best_index_])
print('best_score_:',model.best_score_)
print('scorer_:',model.scorer_)
print('n_splits_:',model.n_splits_)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default 

Attrabutes:vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
cv_results_: dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_C', 'param_kernel', 'param_degree', 'param_gamma', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score', 'split0_train_score', 'split1_train_score', 'split2_train_score', 'split3_train_score', 'mean_train_score', 'std_train_score'])
Desc: {'C': 100, 'kernel': 'linear'} 1.0 0.7875 5
best_estimator_: SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
best_params_: {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
best_params_: {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
best_score_: 0.8775
scorer_: <function _passthrough_scorer at 0x7f4751b7a158>
n_splits_: 4


In [4]:
y_predict_SVM = model.best_estimator_.predict(X_test)
ann_fileName = "SVM_best"+object_user+"_merge_pred_final_rep50.txt"
np.savetxt(ann_fileName, np.array(y_predict_SVM))

In [5]:
#评价指标
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score,accuracy_score

y_true = np.reshape(y_test, [-1])
y_pred = np.reshape(y_predict_SVM, [-1])
  
# 评价指标
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred, average='binary')
f1score = f1_score(y_true, y_pred, average='binary')
  
# Micro F1: 将n分类的评价拆成n个二分类的评价，将n个二分类评价的TP、FP、RN对应相加，计算评价准确率和召回率，由这2个准确率和召回率计算的F1 score即为Micro F1。
# Macro F1: 将n分类的评价拆成n个二分类的评价，计算每个二分类的F1 score，n个F1 score的平均值即为Macro F1。
# 一般来讲，Macro F1、Micro F1高的分类效果好。Macro F1受样本数量少的类别影响大。
micro_f1 = f1_score(y_true, y_pred,average='micro')
macro_f1 = f1_score(y_true, y_pred,average='macro')
  
print('accuracy:',accuracy)
print('precision:',precision)
print('recall:',recall)
print('f1score:',f1score)
print('Macro-F1: {}'.format(macro_f1))
print('Micro-F1: {}'.format(micro_f1))

accuracy: 0.66875
precision: 0.26495726495726496
recall: 0.93
f1score: 0.4124168514412417
Macro-F1: 0.5908907581836321
Micro-F1: 0.66875


神经网络

In [11]:
from numpy.random import seed
seed(1)
tf.random.set_seed(2)

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np
import pickle
import gzip
import matplotlib.pyplot as plt

mlp_clf__tuned_parameters = {"hidden_layer_sizes": [(64,),(64,32),(80,),(80,64),(100,),(100,64),(128,64), (128,)],
                             "solver": ['adam', 'sgd', 'lbfgs'],
                             "max_iter": [20,100,200,1000,5000],
                             "verbose": [True]
                             }
mlp = MLPClassifier()
estimator = GridSearchCV(mlp, mlp_clf__tuned_parameters, n_jobs=6)
estimator.fit(X_train, y_train)

print(estimator.get_params().keys()) 
print (estimator.best_params_)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Iteration 1, loss = 0.73218038
Iteration 2, loss = 0.71826073
Iteration 3, loss = 0.69839369
Iteration 4, loss = 0.67373919
Iteration 5, loss = 0.64845079
Iteration 6, loss = 0.62222147
Iteration 7, loss = 0.59587453
Iteration 8, loss = 0.57119366
Iteration 9, loss = 0.54775865
Iteration 10, loss = 0.52621501
Iteration 11, loss = 0.50601155
Iteration 12, loss = 0.48747587
Iteration 13, loss = 0.47022154
Iteration 14, loss = 0.45483116
Iteration 15, loss = 0.44004512
Iteration 16, loss = 0.42665777
Iteration 17, loss = 0.41460168
Iteration 18, loss = 0.40305177
Iteration 19, loss = 0.39258887
Iteration 20, loss = 0.38284598
Iteration 21, loss = 0.37380643
Iteration 22, loss = 0.36516545
Iteration 23, loss = 0.35746624
Iteration 24, loss = 0.34988691
Iteration 25, loss = 0.34319419
Iteration 26, loss = 0.33658799
Iteration 27, loss = 0.33048402
Iteration 28, loss = 0.32475545
Iteration 29, loss = 0.31928338
Iteration 30, loss = 0.31421720
Iteration 31, loss = 0.30937189
Iteration 32, los

Iteration 255, loss = 0.11317178
Iteration 256, loss = 0.11291318
Iteration 257, loss = 0.11264495
Iteration 258, loss = 0.11238319
Iteration 259, loss = 0.11211853
Iteration 260, loss = 0.11186135
Iteration 261, loss = 0.11159404
Iteration 262, loss = 0.11135177
Iteration 263, loss = 0.11108522
Iteration 264, loss = 0.11082207
Iteration 265, loss = 0.11056288
Iteration 266, loss = 0.11031057
Iteration 267, loss = 0.11005979
Iteration 268, loss = 0.10980504
Iteration 269, loss = 0.10954746
Iteration 270, loss = 0.10929979
Iteration 271, loss = 0.10905142
Iteration 272, loss = 0.10880074
Iteration 273, loss = 0.10855709
Iteration 274, loss = 0.10830389
Iteration 275, loss = 0.10806298
Iteration 276, loss = 0.10781027
Iteration 277, loss = 0.10756227
Iteration 278, loss = 0.10731896
Iteration 279, loss = 0.10708191
Iteration 280, loss = 0.10683667
Iteration 281, loss = 0.10660582
Iteration 282, loss = 0.10636089
Iteration 283, loss = 0.10611828
Iteration 284, loss = 0.10587963
Iteration 

Iteration 512, loss = 0.06780102
Iteration 513, loss = 0.06767974
Iteration 514, loss = 0.06755497
Iteration 515, loss = 0.06743695
Iteration 516, loss = 0.06731777
Iteration 517, loss = 0.06720558
Iteration 518, loss = 0.06709007
Iteration 519, loss = 0.06696370
Iteration 520, loss = 0.06684577
Iteration 521, loss = 0.06673203
Iteration 522, loss = 0.06661110
Iteration 523, loss = 0.06650429
Iteration 524, loss = 0.06638654
Iteration 525, loss = 0.06627024
Iteration 526, loss = 0.06615636
Iteration 527, loss = 0.06604317
Iteration 528, loss = 0.06592605
Iteration 529, loss = 0.06580952
Iteration 530, loss = 0.06569599
Iteration 531, loss = 0.06558090
Iteration 532, loss = 0.06546853
Iteration 533, loss = 0.06535725
Iteration 534, loss = 0.06524352
Iteration 535, loss = 0.06512801
Iteration 536, loss = 0.06501588
Iteration 537, loss = 0.06491764
Iteration 538, loss = 0.06479228
Iteration 539, loss = 0.06468306
Iteration 540, loss = 0.06457231
Iteration 541, loss = 0.06446072
Iteration 

In [17]:
'''
y_predict = model.best_estimator_.predict(X_test)
#y_predict
ann_fileName = "ANN_best"+object_user+"_merge_pred.txt"
np.savetxt(ann_fileName, np.array(y_predict))
'''

In [6]:
from numpy.random import seed
seed(1)
tf.random.set_seed(2)

# two-layer neural network 
# train part
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold
from joblib import dump

#neural network classifier of structure (3,2)
kf = KFold(n_splits=4) # 3-fold cross-validation
best_clf = None
best_score = 0
train_scores = []
test_scores = []
print("kfold-------")
for train_index, test_index in kf.split(X):
    # create neural network using MLPClassifer  {sgd,lbfgs, adam}
    clf = MLPClassifier(solver = 'sgd', max_iter = 1000, hidden_layer_sizes =  (80,),random_state = 1)
    #X_train, X_test = X[train_index], X[test_index]
    #y_train, y_test = y[train_index], y[test_index]
    clf.fit(X_train, y_train)
    train_score = clf.score(X_train, y_train)
    train_scores.append(train_score)
 
    test_score = clf.score(X_test, y_test)
    test_scores.append(test_score)

    #compare score of the tree models and get the best one
    if test_score > best_score:
        best_score = test_score
        best_clf = clf
    
    #print(clf.n_outputs_)
in_sample_error = [1 - score for score in train_scores]
test_set_error = [1 - score for score in test_scores]
print("in_sample_error: ")
print(in_sample_error)
print("test_set_error: ")
print(test_set_error)

#store the classifier
if best_clf != None:
    dump(best_clf, "train_model.m")
    
from joblib import load
clf = load("train_model.m")
y_pred = clf.predict(X_test)
np.savetxt("label_pred.txt", np.array(y_pred)) #save predict result
#print(y_pred)
test_score = clf.score(X_test, y_test)
test_error = 1 - test_score
print('test_score：%s' % test_score)
print('test_error：%s' % test_error)

kfold-------
in_sample_error: 
[0.0050000000000000044, 0.0050000000000000044, 0.0050000000000000044, 0.0050000000000000044]
test_set_error: 
[0.34125000000000005, 0.34125000000000005, 0.34125000000000005, 0.34125000000000005]
test_score：0.65875
test_error：0.34125000000000005


In [14]:
ann_fileName = "ANN_"+object_user+"_merge_pred_final_rep50.txt"
np.savetxt(ann_fileName, np.array(y_pred))

In [7]:
#评价指标
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score,accuracy_score

y_true = np.reshape(y_test, [-1])
y_pred = np.reshape(y_pred, [-1])
  
# 评价指标
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred, average='binary')
f1score = f1_score(y_true, y_pred, average='binary')
  
# Micro F1: 将n分类的评价拆成n个二分类的评价，将n个二分类评价的TP、FP、RN对应相加，计算评价准确率和召回率，由这2个准确率和召回率计算的F1 score即为Micro F1。
# Macro F1: 将n分类的评价拆成n个二分类的评价，计算每个二分类的F1 score，n个F1 score的平均值即为Macro F1。
# 一般来讲，Macro F1、Micro F1高的分类效果好。Macro F1受样本数量少的类别影响大。
micro_f1 = f1_score(y_true, y_pred,average='micro')
macro_f1 = f1_score(y_true, y_pred,average='macro')
  
print('accuracy:',accuracy)
print('precision:',precision)
print('recall:',recall)
print('f1score:',f1score)
print('Macro-F1: {}'.format(macro_f1))
print('Micro-F1: {}'.format(micro_f1))

accuracy: 0.65875
precision: 0.2623626373626374
recall: 0.955
f1score: 0.4116379310344827
Macro-F1: 0.585660514813016
Micro-F1: 0.65875
